In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
import pandas as pd
import re
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import fasttext

In [ ]:
emotes_baus =['SpongeWait', 'bausEZ', 'PETTHEBAUS', 'BRAMBLUS', 'bausPushups', 'KrugsUp', 'BausDonut', 'DaBaus', 'Babush', 'PPogo', 'FeelsHentai', 'SadgeJAM', 'Leanin', 'forsenCD', 'flashAbuser', 'Yoking', 'KKled', 'imposterSus', 'PepeLaffing', 'SirO', 'Svin', 'BABUS', 'PEPW', 'robLick', 'ezChamp', 'Diddles', 'BorpaFoundYou', 'Yoker', 'CLEAN', 'PepeUff', 'peepoChicken', 'vibePLS', 'Chatters', 'WEIRD', 'MODS', 'COCKA', 'DodgeAbusers', 'WeirdChamping', 'JoeBiden', 'TeaTime', 'Shilling', 'YEPCock', 'Weebs', 'IMustLean', 'ClubDance', 'BausBoomer', 'YEPJAM', 'PeterChamp', 'DONUT', 'batJAM', 'DESPAIR', 'lizardPls', 'bausL', 'YEBB', 'BOOBA', 'SadChamp', 'RIOT', 'IRELIAING', 'INSANECAT', 'Cummies', 'Duck', 'SCOOBA', 'Corpa', 'MUTE', 'Boomba', 'ABDULpls', 'MLADY', 'NODDERS', 'peepoHey', 'peepoTalk', 'FeelsWeakMan', 'TriBoom', 'PepegaDriving', 'gachiHYPER', 'Jammies', 'Nerdge', 'borpaSpin', 'KKool', 'peepoRasp', 'pepeMeltdown', 'Peeporiot', 'OOOO', 'PogO', 'PauseChamp', 'Tssk', 'furryRun', 'SION', 'gankAbuser', 'Gaspnt', 'modSheck', 'zhonyasAbuser', 'aniBlush', 'pepeP', 'NOOO', 'FLUSH', 'AYAYA', 'Haha', 'lickR', 'lickL', 'PagWoman', 'SionPassive', 'WeirdShamp', 'EatPoo', 'OMEGALOOOOL', 'Susge', 'Pepeg', 'poopoHappy']
emotes_adept = ['adeptTEA', 'mariRICH', 'peepoRun', 'gachiBASS', 'PepePls', 'Clap', 'pepeJAM', 'EZ', 'HYPERCLAP', 'PepegaPls', 'PepoDance', 'peepoPooPoo', 'pepeD', 'nymnCorn', 'pepeL', 'GuitarTime', 'gachiHYPER', 'ppOverheat', 'FeelsStrongJAM', 'sumSmash', 'COGGERS', 'forsenPls', 'ppHop', 'FeelsRainMan', 'xqcSlam', 'tenseSmash', 'CarlosPls', 'TranceGirls', 'FeelsLagMan', 'TriDance', 'PepegaAim', 'xqcOut', 'pepeMeltdown', 'TeaTime', 'peepoLeave', 'yikesJAM', 'peepoGiggles', 'peepoClap', 'WAYTOODANK', 'OuttaPocket', 'pepeCD', 'HACKERMANS', 'PepegaCredit', 'PepoG', 'PepeMods', 'DonoWall', 'PERIODT', 'BOOBA', 'Drake', 'modCheck', 'monkaSTEER', 'peepoShy', 'peepoFinger', 'PepeLa', 'Jammies', 'NOPERS', 'Chatting', 'Hmmm', 'Copium', 'Madge', 'Prayge', 'CLEAN', 'COWdance', 'DojaDance', 'Awkward', 'BLUBBERS', 'ppSmoke', 'peepoDJ', 'Susge', 'pugPls', 'peepoRiot', 'POLICE', 'lebronJAM', 'NODDERS', 'dojaJAM', 'MeganSus', 'catJAM', 'DinkDonk', 'GIGACHAD', 'HUHH', 'RIPBOZO', 'LOLOL', 'peepoBlonket', 'AlienPls', 'PepeFlushed', 'peepoTalk', 'peepoStir', 'Crying', 'oopTea', 'NOOO', 'pepeW', 'DIESOFCRINGE', 'Sussy', 'VeryPog', 'ppPoof', 'Bedge', 'ddHuh', 'Wokege', 'TomatoTime', 'catAwkward']


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/GregCollab/DataBDA/main/FirstSet.csv')

In [ ]:
data.head()

,Unnamed: 0,Datetime,Usernames,Channels,Messages
0,0,2022-04-26T14:24:31.592519,forever_flat,thebausffs,b'Nerdge'
1,1,2022-04-26T14:24:31.592580,uperplexed,thebausffs,b'SUSSY'
2,2,2022-04-26T14:24:31.750518,johnbellotv,thebausffs,b'REAL VOICE OMEGALUL'
3,3,2022-04-26T14:24:31.750569,y4rrowia,thebausffs,b'LULW'
4,4,2022-04-26T14:24:31.908404,netsubou,thebausffs,b'WutFace'


In [ ]:
#We start by preprocessing the messages. This should be more refined in the future.
messages = data['wo_stop'].str.replace('b\'','') #Due to encoding, each message has a b' infront of it. We remove in this step.
labels = data['Channels'] 
ripper = lambda x: x[:-1] #Remove the ' at the end of the message as well
messages = messages.apply(ripper) 
 #This changes the message <-> channel relation to a word <-> channel relation. This is quite heavy.
splitter = lambda x: x.split() #This splits the message into the words
messages = messages.apply(splitter)
lens = [len(x) for x in messages]
Labeling = lens*(labels + " ").apply(splitter)
messages = [word for sentence in messages for word in sentence] #This splits the message into the words
Labeling = [name for all in Labeling for name in all]
stripper = lambda x: re.sub(r'[^\w]', '', x) #This removes everything that is not a number or a letter (both upper and lower case)
messages = pd.Series(messages).apply(stripper)
messages = messages.str.lower() #This puts everything to on the same case

In [ ]:
data.head()

,Unnamed: 0,Datetime,Usernames,Channels,Messages,wo_stop
0,0,2022-04-26T14:24:31.592519,forever_flat,thebausffs,b'Nerdge',b'Nerdge'
1,1,2022-04-26T14:24:31.592580,uperplexed,thebausffs,b'SUSSY',b'SUSSY'
2,2,2022-04-26T14:24:31.750518,johnbellotv,thebausffs,b'REAL VOICE OMEGALUL',b'REAL VOICE OMEGALUL'
3,3,2022-04-26T14:24:31.750569,y4rrowia,thebausffs,b'LULW',b'LULW'
4,4,2022-04-26T14:24:31.908404,netsubou,thebausffs,b'WutFace',b'WutFace'


In [ ]:
#What we need for the decision tree: The emojis, the estimate from the fasttext and the user
def identify_emotes_baus(text):
    return len([1 for word in emotes_baus if re.search(word, text)])
def identify_emotes_adept(text):
    return len([1 for word in emotes_adept if re.search(word, text)])

def preproc(msg):
    msg = msg.lower()
    msg = re.sub(r'[^\s\w]', '', msg)
    return msg
    
def Pred_probability(msg):
    msg = preproc(msg)
    model  = fasttext.load_model('ft.bin')
    pred = model.predict(msg.lower())[1]
    name = model.predict(msg.lower())[0][0]
    if name == '__label__thebausffs':
        pred = -pred
    else:
        pred = pred
    return float(pred)

In [ ]:
Data = pd.DataFrame({'BausEmote':[], 'AdeptEmote':[], 'Predict':[], 'Username':[], 'Target':[]})
data = data.sample(200)
Data['BausEmote'] = data['Messages'].apply(identify_emotes_baus)
print('Done')
Data['AdeptEmote'] = data['Messages'].apply(identify_emotes_adept)
print('Done')
Data['Predict'] = data['Messages'].apply(Pred_probability)
print('Done')
Data['Username'] = data['Usernames']
Data['Target'] = data['Channels']

Done
Done
Done


In [ ]:
Data['Target']

52501    adeptthebest
19595      thebausffs
11332    adeptthebest
31824      thebausffs
17050      thebausffs
             ...     
8058       thebausffs
4641       thebausffs
35703      thebausffs
42430      thebausffs
48164    adeptthebest
Name: Target, Length: 200, dtype: object

In [ ]:
#train_data, test_data = train_test_split(Data, test_size=0.2)
Data['Target'] = [int(word == 'adeptthebest') for word in Data['Target']]
sparkDF=spark.createDataFrame(Data) 
train_df,test_df= sparkDF.randomSplit([0.5,0.5])

In [ ]:
assembler = VectorAssembler(inputCols=['BausEmote', 'AdeptEmote','Predict'],outputCol="features")
spDF = assembler.transform(sparkDF)
train_df,test_df= spDF.randomSplit([0.8,0.2])

In [ ]:
spDF['features']

Column<'features'>

In [ ]:
from pyspark.ml.classification import LogisticRegressionModel

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr=LogisticRegression(labelCol='Target2',featuresCol='features')
model_lr=lr.fit(train_df)
predictions=model_lr.transform(test_df)
evaluator=BinaryClassificationEvaluator(labelCol="Target2")
accuracy=evaluator.evaluate(predictions)


In [ ]:
model_lr.save('RegModel')

In [ ]:
t = LogisticRegressionModel.load('RegModel')

In [ ]:
from pyspark.sql.functions import udf, struct, array, col, lit


In [ ]:
def Decoder(val):
    if int(val) == 1:
        return 'Adeptthebest'
    else:
        return 'TheBausFFS'
dec_udf = udf(lambda z:Decoder(z))

NameError: name 'df' is not defined

In [ ]:
test_df

DataFrame[BausEmote: bigint, AdeptEmote: bigint, Predict: double, Username: string, Target: bigint, Target2: bigint, features: vector]

In [ ]:
predictions = t.transform(test_df)
evaluator=BinaryClassificationEvaluator(labelCol="Target2")
predictions

DataFrame[BausEmote: bigint, AdeptEmote: bigint, Predict: double, Username: string, Target: bigint, Target2: bigint, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
with open("Data.txt", "w") as output:
    output.write(Data)

TypeError: write() argument must be str, not DataFrame

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.

# Split the data into training and test sets (30% held out for testing)
train_df,test_df= sparkDF.randomSplit([0.8,0.2])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(train_df, numClasses=2, categoricalFeaturesInfo={'BausEmote', 'AdeptEmote', 'Username','Predict'},
                                     impurity='gini', maxDepth=5, maxBins=32)

AssertionError: the data should be RDD of LabeledPoint